In [2]:
import numpy as np # library to handle data in a vectorized manner

import csv

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [17]:
with open('SA_Zipcode.csv', mode='w') as csv_file:
    fieldnames = ['ZIP','B']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

In [26]:
SA_Zipcode = pd.read_csv('SA_Zipcode.csv')
SA_Zipcode.drop(['B','C'], axis=1, inplace=True)
SA_Zipcode

,ZIP
0,78201
1,78202
2,78203
3,78204
4,78205
5,78206
6,78207
7,78208
8,78209
9,78210


In [50]:
def get_geocode(SA_Zipcode):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, San Antonio'.format(Zip))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [41]:
address = 'San Antonio, TX'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Antonio are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Antonio are 29.4246002, -98.4951405.


In [20]:
# create map of San Antonio using latitude and longitude values
map_SA = folium.Map(location=[latitude, longitude], zoom_start=10)
map_SA

In [60]:
CLIENT_ID = 'L0U3IWTNSFH5DWFLD5ONINE3WEM5QDSMSK5K44NAV0KBYZ53' # your Foursquare ID
CLIENT_SECRET = 'V0XFYPTKPHKHRP1ZP1BIJW54FGECRLI1B1LAAHPYDUZ1SG2Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L0U3IWTNSFH5DWFLD5ONINE3WEM5QDSMSK5K44NAV0KBYZ53
CLIENT_SECRET:V0XFYPTKPHKHRP1ZP1BIJW54FGECRLI1B1LAAHPYDUZ1SG2Y


In [174]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 50000 # define radius

query = 'Pizza Places'

# create URL
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    query,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=L0U3IWTNSFH5DWFLD5ONINE3WEM5QDSMSK5K44NAV0KBYZ53&client_secret=V0XFYPTKPHKHRP1ZP1BIJW54FGECRLI1B1LAAHPYDUZ1SG2Y&v=20180605&ll=29.4246002,-98.4951405&query=Pizza Places&radius=50000&limit=1000'

In [175]:
results = requests.get(url).json()

In [176]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [178]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(1000)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Barbaro,Pizza Place,29.455675,-98.491693
1,DOUGH Pizzeria Napoletana,Pizza Place,29.519182,-98.507337
2,Florio's Pizza,Pizza Place,29.499785,-98.467363
3,Main St. Pizza,Pizza Place,29.447439,-98.494090
4,The Station Cafe,Pizza Place,29.417457,-98.490465
5,Pizza Classics,Pizza Place,29.456757,-98.479276
6,Pizza Italia,Pizza Place,29.575156,-98.439420
7,Grimaldi's Pizzeria,Pizza Place,29.495976,-98.478831
8,Stella Public House,Pizza Place,29.409961,-98.495550
9,Alamo Pizza,Pizza Place,29.385755,-98.532876


In [198]:
# create map of SA using latitude and longitude values
map_SA = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SA)  
    
map_SA

In [199]:
# set number of clusters
kclusters = 5

SA_clustering = nearby_venues.drop(nearby_venues.columns[[0,1]], axis=1)
                                                                
                                    
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SA_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 3, 3, 3, 0, 3, 3, 3], dtype=int32)

In [200]:
nearby_venues.insert(0, 'Cluster Labels', kmeans.labels_)

In [201]:
nearby_venues

,Cluster Labels,name,categories,lat,lng
0,3,Barbaro,Pizza Place,29.455675,-98.491693
1,0,DOUGH Pizzeria Napoletana,Pizza Place,29.519182,-98.507337
2,3,Florio's Pizza,Pizza Place,29.499785,-98.467363
3,3,Main St. Pizza,Pizza Place,29.447439,-98.494090
4,3,The Station Cafe,Pizza Place,29.417457,-98.490465
5,3,Pizza Classics,Pizza Place,29.456757,-98.479276
6,0,Pizza Italia,Pizza Place,29.575156,-98.439420
7,3,Grimaldi's Pizzeria,Pizza Place,29.495976,-98.478831
8,3,Stella Public House,Pizza Place,29.409961,-98.495550
9,3,Alamo Pizza,Pizza Place,29.385755,-98.532876


In [203]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters